# Predição de mortalidade em internações por pneumonias bacterianas sensíveis à Atenção Primária no Brasil, 2017-2021

## Análise preditiva

### 1. Importando pacotes de análise

In [87]:
# IMPORTANDO PACOTES 
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd  
import chart_studio.plotly as py
import plotly.graph_objs as gp 
import seaborn as sns
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split

### 2. Importando dados

In [88]:
#IMPOTANDO DADOS
sih_fn = pd.read_csv("sih_pneumonia_1721_trat.csv", index_col= 'N_AIH')
sih_fn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 658719 entries, 1217100049066 to 1721101761060
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   IDADE       658719 non-null  float64
 1   SEXO        658719 non-null  int64  
 2   MORTE       658719 non-null  int64  
 3   DIAS_PERM   658719 non-null  int64  
 4   DIAG_PRINC  658719 non-null  int64  
 5   MARCA_UTI   658719 non-null  int64  
 6   UTI_MES_TO  658719 non-null  int64  
 7   REGIAO      658719 non-null  int64  
dtypes: float64(1), int64(7)
memory usage: 45.2 MB


### 3. Definindo teste e treino

In [89]:
#SEPARANDO OS DADOS
train, test = train_test_split(sih_fn, train_size=0.70, random_state=42)

### 4. Testando algoritmos de classificação

#### 4.1 Regressão Logística

In [90]:
log_reg = smf.logit("MORTE ~ IDADE + SEXO + DIAS_PERM + DIAG_PRINC + MARCA_UTI + UTI_MES_TO + REGIAO", data = train).fit()
print(log_reg.summary())

Optimization terminated successfully.
         Current function value: 0.257790
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                  MORTE   No. Observations:               461103
Model:                          Logit   Df Residuals:                   461095
Method:                           MLE   Df Model:                            7
Date:                Sun, 18 Sep 2022   Pseudo R-squ.:                  0.1776
Time:                        10:47:10   Log-Likelihood:            -1.1887e+05
converged:                       True   LL-Null:                   -1.4453e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -4.8624      0.022   -219.278      0.000      -4.906      -4.819
IDADE          0.0403      0.